In [4]:
from bs4 import BeautifulSoup
import json
import requests
import os
from tqdm import tqdm
from lib import get_search_filename

searches_dir = './searches/'
images_dir = './images/'

In [5]:

def get_soup(post):
    url = post['url']
    soup = BeautifulSoup(requests.get(url).text, 'xml')
    return soup

def get_thumbnail_urls(soup):
    # find the div with id="thumbs"
    thumbs = soup.find('div', id='thumbs')
    # find all the a tags with class="thumb"
    images = thumbs.find_all('a', class_='thumb')

    image_urls = list(map(lambda img: img["href"], images))
    return image_urls


def save_thumbnails(post):
    post_id = post['id']
    image_urls = get_thumbnail_urls(post)

    wrote_images = False
    # save the images to {images_dir}/{post_id}/{i}.jpg
    for i, url in enumerate(image_urls):
        filename = f"{i}.jpg"
        dir_name = os.path.join(images_dir, post_id)
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)
            file_path = os.path.join(dir_name, filename)
            with open(file_path, 'wb') as f:
                f.write(requests.get(url).content)
            wrote_images = True
    return wrote_images

def get_main_image_url(soup):
    # find the div with class="swipe"
    swipe = soup.find('div', class_='swipe')
    # find the img tag
    image = swipe.find('img')
    return image['src']

def save_main_image(image_url):
    # save the image to {images_dir}/{post_id}.jpg
    filename = f"{post_id}.jpg"
    file_path = os.path.join(images_dir, filename)
    wrote_image = False
    if not os.path.exists(file_path):
        with open(file_path, 'wb') as f:
            f.write(requests.get(image_url).content)
        wrote_image = True
    
    return wrote_image

def get_description(soup):
    # get section with id "postingbody"
    postingbody = soup.find('section', id='postingbody')
    # get the text
    description = postingbody.text
    return description

In [7]:
search_file = "limited_100_2022-03-21_17:16:32.json"
with open(os.path.join(searches_dir, search_file), 'r') as f:
    search = json.load(f)


for post_id, post in tqdm(search.items()):
    soup = get_soup(post)
    image_url = get_main_image_url(soup)
    description = get_description(soup)
    post["image_url"] = image_url
    post["description"] = description
    save_main_image(image_url)

filename = get_search_filename("full", len(search))
with open(os.path.join(searches_dir, filename), 'w') as f:
    json.dump(search, f)
    

100%|██████████| 100/100 [00:28<00:00,  3.55it/s]
